In [49]:
def semantic_search_b_node(state: MainState) -> MainState:
    try:
        query = state["user_query"]
        search_results = semantic_search_b(query, top_k=10)
        
        result = [{
            'id': item['id'],
            'similarity': item['similarity_score'],
            'text': item['text'],
            'status': item['status']
        } for item in search_results]
        
        if result:
            print("retrieved document:")
            for item in result:
                print(f"text: {item['text']}")
                print(f"similarity score: {item['similarity']}")
                print(f"status: {item['status']}")
        
        return {"semantic_search_b_results": result}
    except Exception as e:
        print(f"Semantic search B failed: {e}")
        return {"semantic_search_b_results": []}

In [ ]:
def intensity_score(state: MainState) -> MainState:
    try:
        user_query = state["user_query"]
        prompt = (
            f"Analyze sentiment and context of: {user_query}\n\n"
            "Provide sentiment scores (pos, neg, neu) that sum to 1.0\n"
            "Context types: personal (user's feelings), general (about others), question (asking info), academic (educational)\n"
            "Personal relevance: 0.0=impersonal, 1.0=deeply personal"
        )

        llm_response = llm_model.with_structured_output(SentimentScore).invoke(prompt)

        sentiment_scores = {
            'pos': llm_response.pos,
            'neg': llm_response.neg, 
            'neu': llm_response.neu,
            'context_type': llm_response.context_type,
            'personal_relevance': llm_response.personal_relevance
        }
        
        print(f"pos: {llm_response.pos}, neg: {llm_response.neg}, neu: {llm_response.neu}")
        print(f"context: {llm_response.context_type}, relevance: {llm_response.personal_relevance}")
        
        return {"intensity_score": sentiment_scores}
    except:
        print("fail")
        return {"intensity_score": {}}

In [18]:
def top_k_filter(state: MainState) -> MainState:
    try:
        search_results = state.get("semantic_search_b_results", [])
        
        if not search_results:
            print("no search results to filter")
            return {"top_k_results": []}
        
        user_query = state["user_query"]
        prompt = (
            f"Filter mental health search results for relevance to: {user_query}\n\n"
            "Keep only results that directly relate to the user's mental state.\n"
            "Remove off-topic or generic content.\n\n"
            "SEARCH RESULTS:\n"
        )
        
        # Limit to first 5 results to avoid token limits
        for i, result in enumerate(search_results[:5], 1):
            prompt += (
                f"{i}. ID: {result['id']}\n"
                f"   Similarity: {result['similarity']:.3f}\n"
                f"   Status: {result['status']}\n"
                f"   Text: {result['text'][:200]}...\n\n"
            )
        
        llm_response = llm_model.with_structured_output(List[FilteredResult]).invoke(prompt)
        
        filtered_results = [
            {
                "id": result.id,
                "similarity": result.similarity,
                "status": result.status,
                "text": result.text
            }
            for result in llm_response
        ]
        
        print(f"filtered {len(filtered_results)} from {len(search_results)} results")
        
        return {"top_k_results": filtered_results}
    except Exception as e:
        print("filter fail")
        return {"top_k_results": []}

In [19]:
def merge_path_B(state: MainState) -> MainState:
    intensity_score = state.get("intensity_score", {})
    top_k_results = state.get("top_k_results", [])
    
    # Check what data we have
    if intensity_score and top_k_results:
        print("merged!")
    elif not intensity_score and not top_k_results:
        print("intensity-score: null")
        print("top_k: null")
    elif not intensity_score:
        print("intensity-score: null")
    elif not top_k_results:
        print("top_k: null")
    
    # Simple passthrough - no new fields needed
    return {}

In [20]:
def calculator_func(state: MainState) -> MainState:
    try:
        top_k_results = state.get("top_k_results", [])
        
        # Fix field names to match calculator expectations
        fixed_top_k = []
        for result in top_k_results:
            fixed_result = {
                "similarity_score": result["similarity"],  # Fix field name
                "label": result["status"],                 # Fix field name  
                "text": result["text"],
                "id": result["id"]
            }
            fixed_top_k.append(fixed_result)
        
        intensity_score = state.get("intensity_score", {
            "pos": 0.33, "neg": 0.33, "neu": 0.34,
            "context_type": "personal", "personal_relevance": 1.0
        })
        
        current_scores = state.get("user_scores")
        decay_scores = state.get("user_decay_scores") 
        last_update = state.get("last_update_timestamp")
        
        # Calculate updated scores
        updated_scores, updated_decay, timestamp, calc_result = calculator.calculate_scores(
            user_id="temp",  # Not used in calculation logic
            top_k_results=fixed_top_k,
            intensity_score=intensity_score,
            current_scores=current_scores,
            decay_scores=decay_scores,
            last_update_timestamp=last_update
        )
        
        # Validate calc_result
        calc_result = max(0.0, min(100.0, calc_result))
        
        print(f"calc_result: {calc_result:.2f}")
        
        return {
            "user_scores": updated_scores,
            "user_decay_scores": updated_decay,
            "last_update_timestamp": timestamp,
            "calc_result": calc_result
        }
    except Exception as e:
        print(f"calculator fail: {e}")
        return {
            "user_scores": state.get("user_scores", {}),
            "user_decay_scores": state.get("user_decay_scores", {}),
            "last_update_timestamp": state.get("last_update_timestamp"),
            "calc_result": 0.0
        }

In [21]:
def warning_gen_flag(state: MainState) -> MainState:
    try:
        calc_result = state.get("calc_result", 0.0)
        user_scores = state.get("user_scores", {})
        
        if not user_scores:
            print("no warning - no scores")
            return {"warning_text": ""}
        
        # Check for critical conditions first (suicide risk)
        suicidal_score = user_scores.get("Suicidal", 0.0)
        if suicidal_score > 70.0:  # High suicide risk regardless of calc_result
            print(f"critical warning - suicidal: {suicidal_score:.1f}")
            return {"warning_text": f"Critical concern detected: Suicidal indicators ({suicidal_score:.1f}/100)"}
        
        # Standard threshold-based warning (lowered to 30 for better sensitivity)
        if calc_result >= 30.0:
            # Get concerning scores (excluding Normal, above 15/100)
            negative_scores = [
                (label, score) for label, score in user_scores.items() 
                if label != "Normal" and score > 15.0
            ]
            
            # Sort by highest scores and get top 2
            top_2 = sorted(negative_scores, key=lambda x: x[1], reverse=True)[:2]
            
            if len(top_2) >= 2:
                warning_text = f"Elevated indicators: {top_2[0][0]} ({top_2[0][1]:.1f}/100), {top_2[1][0]} ({top_2[1][1]:.1f}/100)"
            elif len(top_2) == 1:
                warning_text = f"Elevated indicators: {top_2[0][0]} ({top_2[0][1]:.1f}/100)"
            else:
                warning_text = "General concern detected"
            
            print(f"warning generated - calc: {calc_result:.1f}")
            return {"warning_text": warning_text}
        
        print("no warning generated")
        return {"warning_text": ""}
        
    except Exception as e:
        print(f"warning fail: {e}")
        return {"warning_text": ""}